In [73]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [118]:
labels=pd.read_csv('labels.csv', encoding='utf-8')

In [92]:
labels['Year'] = pd.to_datetime(labels['commit_date']).dt.year
labels['commit_date'] = pd.to_datetime(labels['commit_date'])

labels_2015 = labels[labels['Year'] == 2015]
labels_2016 = labels[labels['Year'] == 2016]
labels_2017=  labels[labels['Year']== 2017]
labels_2018 = labels[labels['Year'] == 2018]
labels_2019 = labels[labels['Year'] == 2019]
labels_2020 = labels[labels['Year'] == 2020]
labels_2021 = labels[labels['Year'] == 2021]
labels_2022 = labels[labels['Year'] == 2022]


In [ ]:
pip install imblearn

In [119]:
labels.drop (['info_version','avg_cosine_sim','commit_date'], axis=1, inplace=True)

In [120]:
labels.fillna(0, inplace=True)

In [96]:
# Oversampling using SMOTE
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.model_selection import train_test_split

from imblearn.under_sampling import RandomUnderSampler

X = labels.drop(['api_spec_id', 'label', 'externalDocs.added', 'externalDocs.deleted','cosine_sim'], axis=1)
y = labels['label']   # target variable

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, shuffle=True, stratify=y)

oversampled = SMOTE(random_state=0,
                    k_neighbors=4)
X_train_smote, y_train_smote = oversampled.fit_resample(X_train, y_train)

In [97]:
y_train_smote = pd.DataFrame(y_train_smote , columns=['label'])
y_train_smote

,label
0,minor
1,major
2,minor
3,patch
4,patch
...,...
8704,minor
8705,minor
8706,minor
8707,minor


In [81]:
y_test = pd.DataFrame(y_test , columns=['label'])

In [42]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()

# Get columns to use
cols_to_use = X_train_smote[['Endpoints_added','Components_parameters_added','Paths_added', 'Paths_deleted', 'Tags_deleted', 'Components_Schemas_Added', 'Info_description_changes', 'Tags_modified', 'Paths_modified', 'Endpoints_modified']].columns

data_scaled = scaler.fit_transform(X_train_smote[cols_to_use])

# Define the model
model = GaussianMixture(n_components=3, random_state=32, max_iter=200, init_params='random', covariance_type='full', n_init=1, warm_start=True)

# Fit the model
model.fit(data_scaled)

# Get the cluster labels
cluster_labels = model.predict(data_scaled)

# Map the cluster labels to major, minor, and patch labels
label_map = {0: "major", 1: "minor", 2: "patch"}
predicted_labels = [label_map[label] for label in cluster_labels]

# Add the major, minor, and patch labels to the dataframe
y_train_smote['predicted_labels'] = predicted_labels

# Get the accuracy score
accuracy_score(y_train_smote['label'], predicted_labels)

# Get the precision score
precision_score(y_train_smote['label'], predicted_labels, average='weighted')

# Get the recall score
recall_score(y_train_smote['label'], predicted_labels, average='weighted')

# Get the f1 score
f1_score(y_train_smote['label'], predicted_labels, average='weighted')

# Get the classification report
print(classification_report(y_train_smote['label'], predicted_labels))



              precision    recall  f1-score   support

       major       0.26      0.02      0.04      2889
       minor       0.60      0.26      0.37      2889
       patch       0.34      0.85      0.49      2889

    accuracy                           0.38      8667
   macro avg       0.40      0.38      0.30      8667
weighted avg       0.40      0.38      0.30      8667



In [82]:
# define a dictionary that maps labels to numbers
label_to_number = {'major': 0, 'minor': 1, 'patch': 2}

# use the map() method to apply the mapping to the label column
y_test['label'] = y_test['label'].map(label_to_number)

In [47]:
y_train_smote['label'].groupby(y_train_smote['predicted_labels']).value_counts()

predicted_labels  label
major             minor     112
                  patch      61
                  major      60
minor             minor     764
                  patch     361
                  major     151
patch             major    2678
                  patch    2467
                  minor    2013
Name: label, dtype: int64

In [122]:
from sklearn.model_selection import train_test_split

X = labels.drop(['api_spec_id', 'label', 'externalDocs.added', 'externalDocs.deleted','cosine_sim'], axis=1)
y = labels['label']   # target variable

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, shuffle=True, stratify=y)


X_train = pd.DataFrame(X_train , columns= X_train.columns)
y_train = pd.DataFrame(y_train , columns=['label'])


In [130]:
scaler = StandardScaler()

# Get columns to use
cols_to_use = X_train[['Endpoints_added', 'Endpoints_deleted','Components_parameters_added', 'Paths_added', 'Paths_deleted', 'Components_Schemas_Added', 'Components_Schemas_Deleted', 'Info_description_changes', 'Paths_modified', 'Endpoints_modified']].columns

data_scaled = scaler.fit_transform(X_train[cols_to_use])

# Define the model
model = GaussianMixture(n_components=3, random_state=32, max_iter=200, init_params='random', covariance_type='full', n_init=1, warm_start=True)

# Fit the model
model.fit(data_scaled)

# Get the cluster labels
cluster_labels = model.predict(data_scaled)

# Map the cluster labels to major, minor, and patch labels
label_map = {0: "major", 1: "minor", 2: "patch"}
predicted_labels = [label_map[label] for label in cluster_labels]

# Add the major, minor, and patch labels to the dataframe
y_train['predicted_labels'] = predicted_labels

# Get the accuracy score
accuracy_score(y_train['label'], predicted_labels)

# Get the precision score
precision_score(y_train['label'], predicted_labels, average='weighted')

# Get the recall score
recall_score(y_train['label'], predicted_labels, average='weighted')

# Get the f1 score
f1_score(y_train['label'], predicted_labels, average='weighted')

# Get the classification report
print(classification_report(y_train['label'], predicted_labels))



              precision    recall  f1-score   support

       major       0.03      0.05      0.04       449
       minor       0.19      0.68      0.30       932
       patch       0.37      0.04      0.07      2903

    accuracy                           0.18      4284
   macro avg       0.20      0.26      0.14      4284
weighted avg       0.29      0.18      0.12      4284



In [105]:
from sklearn.model_selection import train_test_split

X = labels.drop(['api_spec_id', 'label', 'externalDocs.added', 'externalDocs.deleted','cosine_sim','commit_date','avg_cosine_sim','predicted_labels','info_version'], axis=1)
y = labels['label']   # target variable

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, shuffle=True, stratify=y)



In [85]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import warnings

pd.options.mode.chained_assignment = None


# load subset of dataframes into list dfs
dfs = [labels_2015, labels_2016, labels_2017, labels_2018, labels_2019, labels_2020, labels_2021, labels_2022]

for i, df in enumerate(dfs):
    scaler = StandardScaler()

    # Get columns to use
    cols_to_use = df[['Endpoints_added', 'Components_parameters_added', 'Paths_added', 'Paths_deleted', 'Tags_deleted', 'Components_Schemas_Added', 'Info_description_changes', 'Tags_modified', 'Paths_modified', 'Endpoints_modified']].columns

    data_scaled = scaler.fit_transform(df[cols_to_use])

    # Define the model
    model = GaussianMixture(n_components=3, random_state=32, max_iter=200, init_params='random', covariance_type='full', n_init=1, warm_start=True)

    # Fit the model
    model.fit(data_scaled)

    # Get the cluster labels
    cluster_labels = model.predict(data_scaled)

    # Map the cluster labels to major, minor, and patch labels
    label_map = {0: "major", 1: "minor", 2: "patch"}
    predicted_labels = [label_map[label] for label in cluster_labels]

    # Add the major, minor, and patch labels to the dataframe
    df["predicted_labels"] = predicted_labels

    # Accuracy
    print("Accuracy for dataframe", i+1)
    print(accuracy_score(df["label"], df["predicted_labels"]))
    
    # Precision
    print("Precision for dataframe", i+1)
    print(precision_score(df["label"], df["predicted_labels"], average="weighted"))

    # Recall
    print("Recall for dataframe", i+1)
    print(recall_score(df["label"], df["predicted_labels"], average="weighted"))

    # F1
    print("F1 for dataframe", i+1)
    print(f1_score(df["label"], df["predicted_labels"], average="weighted"))

    # Classification report
    print(classification_report(df["label"], df["predicted_labels"]))



Accuracy for dataframe 1
0.1875
Precision for dataframe 1
0.05000000000000001
Recall for dataframe 1
0.1875
F1 for dataframe 1
0.07894736842105264
              precision    recall  f1-score   support

       major       0.00      0.00      0.00        35
       minor       0.20      0.75      0.32        12
       patch       0.00      0.00      0.00         1

    accuracy                           0.19        48
   macro avg       0.07      0.25      0.11        48
weighted avg       0.05      0.19      0.08        48

Accuracy for dataframe 2
0.3333333333333333
Precision for dataframe 2
0.21999726999727
Recall for dataframe 2
0.3333333333333333
F1 for dataframe 2
0.2534939553573094
              precision    recall  f1-score   support

       major       0.13      0.06      0.08        34
       minor       0.00      0.00      0.00        27
       patch       0.40      0.70      0.51        50

    accuracy                           0.33       111
   macro avg       0.18      0.25

In [90]:
# Compute confusion matrix for each dataframe using plotly

from sklearn.metrics import confusion_matrix
import seaborn as sns
import plotly.figure_factory as ff
import plotly.graph_objects as go

for i, df in enumerate(dfs):
    # Compute confusion matrix
    cm = confusion_matrix(df["label"], df["predicted_labels"])

    # Create heatmap
    fig = ff.create_annotated_heatmap(cm, x=['major', 'minor', 'patch'], y=['major', 'minor', 'patch'], colorscale='twilight')

    # Add title
    fig.update_layout(
    title_text="Confusion matrix for Dataframe {}".format(i+1),
    xaxis=dict(title='Predicted Labels'),
    yaxis=dict(title='True Labels')
)
    # Show heatmap
    fig.show()
